# Open data

In [6]:
text = """
– Дяденька Король, Цветной Капусты хотца, – попросил он, покачиваясь на конце ветки у самого окна королевской спальни.
Королева от возмущения упала в обморок, а Король успел поднять стражу, которая оцепила морковный дуб, предлагая крольчонку сдаться живым, а в крайнем случае, мертвым. Крольчонок ничего не отвечал, но время от времени с неряшливой меткостью бросал в стражников совершенно несъедобные, однако довольно увесистые морковные желуди.
Часть стражников была тяжело ранена, зато остальные пришли в ярость и, уже осыпаемые ядрами морковных желудей, штурмом овладели этой неожиданной цитаделью, как впоследствии писали королевские историки.
Стражники облазили все ветки, но крольчонка нигде не оказалось. Тогда они, решив, что он замаскировался в листве дуба, стали поочередно трясти все ветки, растягивая под каждой из них сетку из пампасской травы.
Еще несколько стражников было ранено своими же трясунами, и наконец некое легкое тело свалилось в сетку и запуталось в ней.
Но, увы, Король, вышедший посмотреть на возмутителя королевства, был еще более удручен. Мало того что, пока он выходил из дворца и приближался к морковному дубу, мимо него пронесли около тридцати тяжелораненых стражников, но, когда он подошел к сетке и ее осторожно распутали, в ней оказалась белка.
– Ничего, мы доберемся до его кроличьей шкуры, – сказал Начальник Королевской Охраны и велел осторожно вместе с сеткой внести белку в помещение для допросов провинившихся кроликов.
– Еще один такой штурм – и я останусь без армии, – сказал Король, горестно и брезгливо оглядывая место сражения.
Дело в том, что в королевстве кроликов Охрана Короля была равнозначна охране королевства и, естественно, считалась армией. Армия была вооружена бамбуковыми пиками, бамбуковыми палками и бамбуковыми трубками, выстреливающими кактусовой иглой. Убойная сила стреляющей трубки была равна среднему попугаю, но не годилась ни против шкуры туземцев, ни тем более против шкуры удавов.
"""
X_train = [' '.join(text.split()[i:i+5]) for i in range(0, len(text.split()), 5)]
len(X_train), X_train[:5]


(57,
 ['– Дяденька Король, Цветной Капусты',
  'хотца, – попросил он, покачиваясь',
  'на конце ветки у самого',
  'окна королевской спальни. Королева от',
  'возмущения упала в обморок, а'])

# Define grid search pipeline

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import math
import numpy as np
from utils.preprocessor import Preprocessor
from sklearn.metrics import silhouette_score

def c(n, k):
    """
    k combinations from n
    """
    return math.factorial(n) / (math.factorial(n-k) * math.factorial(k))

def clf_search(pipe, clf_name):
    for n, i in enumerate(pipe.steps):
        param = clf_name.split('__')
        if i[0] == param[0]:
            return n, param[1]
               
parameters = { 
    'tfidf__max_df': (0.9, .95, .8, .85),
    'tfidf__min_df': (0., 0.01, 0.05),
    'tfidf__max_features': (100, 200, 500),
    'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2), (3, 3), (1, 3)],
    'kmeans__n_clusters': [20, 15, 10, 5]
}

pipeline = Pipeline([
    ('preproc', Preprocessor(infinitive_mode='no')),
    ('tfidf', TfidfVectorizer(analyzer='word', max_features=100)),
    ('kmeans', KMeans(random_state=42))
])

# set default to best. Take only names that take place in parameters dict
best_params_set = {clf_search(pipeline, k)[1]:pipeline.steps[clf_search(pipeline, k)[0]][1].get_params()[clf_search(pipeline, k)[1]] for k, v in parameters.items()}

# default value for target metric
current_s_score = -1

# grid_search or beam_search strategy for parameters search
# beam_search currently doesnt work properly! It takes all combinations insted of taking one within one param
grid_search = True
beam_search = False if grid_search else True

if grid_search:
    steps = sum([len(i) for i in parameters.values()])
else:
    params_amaount_to_test = sum([len(i) for i in parameters.values()])
    steps = sum([c(params_amaount_to_test, i+1) for i in range(params_amaount_to_test)])    
step = 1


# Run gridsearch

In [11]:
for k, v in parameters.items():
    for value in v:
        print(f'step {step}/{steps}')
        clf_id, param_name = clf_search(pipeline, k)
        default_params = pipeline.steps[clf_id][1].get_params()
        current_param = {param_name: value}
        pipeline.steps[clf_id][1].set_params(**current_param)
        
        # fit
        pipeline.fit(X_train)
        
        # estimate
        vectors = np.asarray(pipeline.steps[1][1].transform(X_train).todense())
        centroids = pipeline.steps[2][1].cluster_centers_
        new_labels = pipeline.steps[2][1].labels_
        
        # evaluate
        s_score = silhouette_score(vectors, 
                                 new_labels)
        if s_score > current_s_score:
            current_s_score = s_score
            best_params_set[k.split('__')[1]] = value
        print('silhouette_score', s_score, current_param)
        pipeline.steps[clf_id][1].set_params(**default_params)
        step += 1
print(current_s_score, best_params_set)

step 1/19
silhouette_score 0.06991759545565934 {'max_df': 0.9}
step 2/19
silhouette_score 0.06991759545565934 {'max_df': 0.95}
step 3/19
silhouette_score 0.06991759545565934 {'max_df': 0.8}
step 4/19
silhouette_score 0.06991759545565934 {'max_df': 0.85}
step 5/19
silhouette_score 0.06991759545565934 {'min_df': 0.0}
step 6/19
silhouette_score 0.06991759545565934 {'min_df': 0.01}
step 7/19
silhouette_score 1.0 {'min_df': 0.05}
step 8/19
silhouette_score 0.06991759545565934 {'max_features': 100}
step 9/19


/Users/georgychernousov/miniconda3/envs/snakes/lib/python3.8/site-packages/sklearn/pipeline.py:346: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


silhouette_score 0.03888343904539529 {'max_features': 200}
step 10/19
silhouette_score 0.03888343904539529 {'max_features': 500}
step 11/19
silhouette_score 0.06991759545565934 {'ngram_range': (1, 1)}
step 12/19
silhouette_score 0.12256026770668725 {'ngram_range': (1, 2)}
step 13/19
silhouette_score -0.3476626501423776 {'ngram_range': (2, 2)}
step 14/19
silhouette_score -0.44335476589076284 {'ngram_range': (3, 3)}
step 15/19
silhouette_score 0.17395619252212438 {'ngram_range': (1, 3)}
step 16/19
silhouette_score 0.07425629966425072 {'n_clusters': 20}
step 17/19
silhouette_score 0.0696678207172972 {'n_clusters': 15}
step 18/19
silhouette_score 0.08807361734091579 {'n_clusters': 10}
step 19/19
silhouette_score 0.05539593506799961 {'n_clusters': 5}
1.0 {'max_df': 0.9, 'min_df': 0.05, 'max_features': 100, 'ngram_range': (1, 1), 'n_clusters': 8}
